Wilcoxon test

In [1]:
import pandas as pd
import numpy as np
import CRPS.CRPS as pscore
from KDEpy.bw_selection import silvermans_rule
import random

random.seed(42)
np.random.seed(42)

In [2]:
def calculate_pointwise_rmse(actual, pred):
    if len(pred.shape) > 1:
        pred = pred[:, 9]  # 取中位数列
    return np.sqrt((actual - pred) ** 2)


def calculate_pointwise_mae(actual, pred):
    if len(pred.shape) > 1:
        pred = pred[:, 9]  # 取中位数列
    return np.abs(actual - pred)


def calculate_pointwise_crps(actual, pred, sample_size=100):

    n = len(actual)
    crps_values = []

    std = [silvermans_rule(pred[i, :].reshape(-1, 1)) for i in range(n)]

    for i in range(n):
        resampled = np.random.choice(pred[i, :], size=sample_size, replace=True)
        resampled += np.random.randn(sample_size) * std[i]

        crps_val = pscore(resampled, actual[i]).compute()[0]
        crps_values.append(crps_val)

    return np.array(crps_values)

In [3]:
def process_all_models():
    """处理所有模型并保存到一个Excel文件"""

    # 读取实际值
    print("Loading actual values...")
    actual_path = 'C:/Users/ym_ya/Desktop/windpower/Simulation code and data/section 4/actual.xlsx'
    actual = pd.read_excel(actual_path)
    actual = np.squeeze(np.array(actual))

    # 定义所有模型
    models = {
        'TRMG': 'C:/Users/ym_ya/Desktop/windpower/Simulation code and data/trmg/trmg.xlsx',
        'TRQG': 'C:/Users/ym_ya/Desktop/windpower/Simulation code and data/trqg/trqg.xlsx',
        'WRGG': 'C:/Users/ym_ya/Desktop/windpower/Simulation code and data/wrgg/wrgg.xlsx',
        'VRLG': 'C:/Users/ym_ya/Desktop/windpower/Simulation code and data/vrlg/vrlg.xlsx',
        'ERMG': 'C:/Users/ym_ya/Desktop/windpower/Simulation code and data/ermg/ermg.xlsx',
        'TSMG': 'C:/Users/ym_ya/Desktop/windpower/Simulation code and data/tsmg/tsmg.xlsx',
        'QRNN2': 'C:/Users/ym_ya/Desktop/windpower/Simulation code and data/qrnn2/qrnn2pred.xlsx',
        'MCQRNN': 'C:/Users/ym_ya/Desktop/windpower/Simulation code and data/mcqrnn/mcqrnn2pred.xlsx',
        'QRF': 'C:/Users/ym_ya/Desktop/windpower/Simulation code and data/qrf/predqrf.xlsx',
        'QRLASSO': 'C:/Users/ym_ya/Desktop/windpower/Simulation code and data/qrlasso/lassopred.xlsx',
    }

    # 输出文件路径
    output_path = 'C:/Users/ym_ya/Desktop/windpower/pointwise_metrics.xlsx'

    # 创建Excel writer
    print(f"\nProcessing {len(models)} models...")
    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:

        for idx, (model_name, pred_path) in enumerate(models.items(), 1):
            print(f"\n[{idx}/{len(models)}] Processing {model_name}...")

            try:
                # 读取预测值
                pred = pd.read_excel(pred_path)
                pred = np.array(pred)

                # 计算逐点指标
                print(f"  - Calculating RMSE...")
                rmse_pointwise = calculate_pointwise_rmse(actual, pred)

                print(f"  - Calculating MAE...")
                mae_pointwise = calculate_pointwise_mae(actual, pred)

                print(f"  - Calculating CRPS...")
                crps_pointwise = calculate_pointwise_crps(actual, pred)

                # 创建结果DataFrame
                results_df = pd.DataFrame({
                    'Point_Index': range(len(actual)),
                    'Actual': actual,
                    'Predicted': pred[:, 9] if len(pred.shape) > 1 else pred,
                    'RMSE': rmse_pointwise,
                    'MAE': mae_pointwise,
                    'CRPS': crps_pointwise
                })


                # 写入Excel的不同sheet
                sheet_name = model_name[:31]  # Excel sheet名称限制31字符
                results_df.to_excel(writer, sheet_name=sheet_name, index=False)
                print(f"  ✓ Saved to sheet '{sheet_name}'")

            except Exception as e:
                print(f"  ✗ Error processing {model_name}: {str(e)}")
                continue

    print(f"\n{'='*60}")
    print(f"All results saved to:")
    print(f"{output_path}")
    print(f"{'='*60}")

    return output_path

In [4]:
if __name__ == "__main__":
    output_file = process_all_models()

Loading actual values...

Processing 10 models...

[1/10] Processing TRMG...
  - Calculating RMSE...
  - Calculating MAE...
  - Calculating CRPS...
  ✓ Saved to sheet 'TRMG'

[2/10] Processing TRQG...
  - Calculating RMSE...
  - Calculating MAE...
  - Calculating CRPS...
  ✓ Saved to sheet 'TRQG'

[3/10] Processing WRGG...
  - Calculating RMSE...
  - Calculating MAE...
  - Calculating CRPS...
  ✓ Saved to sheet 'WRGG'

[4/10] Processing VRLG...
  - Calculating RMSE...
  - Calculating MAE...
  - Calculating CRPS...
  ✓ Saved to sheet 'VRLG'

[5/10] Processing ERMG...
  - Calculating RMSE...
  - Calculating MAE...
  - Calculating CRPS...
  ✓ Saved to sheet 'ERMG'

[6/10] Processing TSMG...
  - Calculating RMSE...
  - Calculating MAE...
  - Calculating CRPS...
  ✓ Saved to sheet 'TSMG'

[7/10] Processing QRNN2...
  - Calculating RMSE...
  - Calculating MAE...
  - Calculating CRPS...
  ✓ Saved to sheet 'QRNN2'

[8/10] Processing MCQRNN...
  - Calculating RMSE...
  - Calculating MAE...
  - 

C:\Users\ym_ya\AppData\Local\Programs\Python\Python312\Lib\site-packages\KDEpy\bw_selection.py:285: UserWarning: Silverman's rule failed. Too many idential values. Setting bw = 0.1156443191928751
  warnings.warn(
C:\Users\ym_ya\AppData\Local\Programs\Python\Python312\Lib\site-packages\KDEpy\bw_selection.py:285: UserWarning: Silverman's rule failed. Too many idential values. Setting bw = 0.17977072141268838
  warnings.warn(
C:\Users\ym_ya\AppData\Local\Programs\Python\Python312\Lib\site-packages\KDEpy\bw_selection.py:285: UserWarning: Silverman's rule failed. Too many idential values. Setting bw = 0.2472930763722718
  warnings.warn(
C:\Users\ym_ya\AppData\Local\Programs\Python\Python312\Lib\site-packages\KDEpy\bw_selection.py:285: UserWarning: Silverman's rule failed. Too many idential values. Setting bw = 0.2172159011950012
  warnings.warn(
C:\Users\ym_ya\AppData\Local\Programs\Python\Python312\Lib\site-packages\KDEpy\bw_selection.py:285: UserWarning: Silverman's rule failed. Too many 

  ✓ Saved to sheet 'QRF'

[10/10] Processing QRLASSO...
  - Calculating RMSE...
  - Calculating MAE...
  - Calculating CRPS...
  ✓ Saved to sheet 'QRLASSO'

All results saved to:
C:/Users/ym_ya/Desktop/windpower/pointwise_metrics.xlsx


In [1]:
from scipy.stats import shapiro, wilcoxon
import warnings
import pandas as pd
import numpy as np
warnings.filterwarnings('ignore')

input_file = 'C:/Users/ym_ya/Desktop/windpower/Simulation code and data/5.2/pointwise_metrics.xlsx'

metrics = ['RMSE', 'MAE', 'CRPS']

In [2]:
all_results = {
    'shapiro': {},
    'wilcoxon': {}
}

for metric in metrics:
    print(f"\n{'='*80}")
    print(f"Metric: {metric}")
    print(f"{'='*80}")

    sheet_names = ['TRMG', 'TRQG', 'WRGG', 'VRLG', 'ERMG', 'TSMG', 'QRNN2', 'MCQRNN', 'QRF', 'QRLASSO']

    model_data = {}
    for sheet in sheet_names:
        df = pd.read_excel(input_file, sheet_name=sheet)

        valid_rows = []
        for idx, row in df.iterrows():
            if pd.isna(row['Point_Index']) or row['Point_Index'] == '' or row['Point_Index'] == 'Mean':
                break
            valid_rows.append(idx)
        model_data[sheet] = df.loc[valid_rows, metric].values

    baseline_model = 'TRMG'
    baseline_data = model_data[baseline_model]

    shapiro_results = []
    wilcoxon_results = []

    print(f"\n[Step 1] Shapiro-Wilk Normality Test for Differences")
    print("-" * 80)
    print(f"Testing if (TRMG - Other_Model) follows normal distribution")
    print(f"H0: Data follows normal distribution")
    print(f"H1: Data does NOT follow normal distribution")
    print(f"Significance level: α = 0.05")
    print("-" * 80)

    for model_name in sheet_names:
        if model_name == baseline_model:
            continue

        diff = baseline_data - model_data[model_name]

        stat_shapiro, p_shapiro = shapiro(diff)

        is_normal = "Parametric (Normal)" if p_shapiro > 0.05 else "Non-parametric (Not Normal)"

        shapiro_results.append({
            'Comparison': f'TRMG - {model_name}',
            'Statistic': stat_shapiro,
            'P-value': p_shapiro,
            'Distribution': is_normal,
            'Reject H0': 'Yes' if p_shapiro <= 0.05 else 'No'
        })

        print(f"\nTRMG vs {model_name}:")
        print(f"  Shapiro-Wilk Statistic: {stat_shapiro:.6f}")
        print(f"  P-value: {p_shapiro:.6f}")
        print(f"  Distribution: {is_normal}")
        print(f"  Reject H0 (not normal): {'Yes' if p_shapiro <= 0.05 else 'No'}")

    shapiro_df = pd.DataFrame(shapiro_results)
    all_results['shapiro'][metric] = shapiro_df

    print(f"\n[Step 2] Wilcoxon Signed-Rank Test")
    print("-" * 80)
    print(f"Testing if there is significant difference between TRMG and other models")
    print(f"H0: Median difference = 0 (no significant difference)")
    print(f"H1: Median difference ≠ 0 (significant difference)")
    print(f"Significance level: α = 0.05")
    print("-" * 80)

    for model_name in sheet_names:
        if model_name == baseline_model:
            continue

        diff = baseline_data - model_data[model_name]

        stat_wilcoxon, p_wilcoxon = wilcoxon(diff, alternative='two-sided')

        is_significant = "Significant" if p_wilcoxon <= 0.05 else "Not Significant"

        median_diff = np.median(diff)
        if p_wilcoxon <= 0.05:
            if median_diff > 0:
                performance = "TRMG is significantly WORSE"
            else:
                performance = "TRMG is significantly BETTER"
        else:
            performance = "No significant difference"

        wilcoxon_results.append({
            'Comparison': f'TRMG vs {model_name}',
            'Statistic': stat_wilcoxon,
            'P-value': p_wilcoxon,
            'Median Diff': median_diff,
            'Significance': is_significant,
            'Reject H0': 'Yes' if p_wilcoxon <= 0.05 else 'No',
            'Interpretation': performance
        })

        print(f"\nTRMG vs {model_name}:")
        print(f"  Wilcoxon Statistic: {stat_wilcoxon:.2f}")
        print(f"  P-value: {p_wilcoxon:.6f}")
        print(f"  Median Difference: {median_diff:.3f}")
        print(f"  Significance: {is_significant}")
        print(f"  Interpretation: {performance}")

    wilcoxon_df = pd.DataFrame(wilcoxon_results)
    all_results['wilcoxon'][metric] = wilcoxon_df

# 保存所有结果到Excel
output_file = 'statistical_tests_results.xlsx'

print(f"\n{'='*80}")
print("Saving results to Excel...")
print(f"{'='*80}")

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    # 为每个指标创建两个sheet
    for metric in metrics:
        # Shapiro-Wilk结果
        shapiro_sheet = f'{metric}_Shapiro'
        all_results['shapiro'][metric].to_excel(writer, sheet_name=shapiro_sheet, index=False)

        # Wilcoxon结果
        wilcoxon_sheet = f'{metric}_Wilcoxon'
        all_results['wilcoxon'][metric].to_excel(writer, sheet_name=wilcoxon_sheet, index=False)

print(f"✓ Results saved to: {output_file}")

# 打印汇总
print(f"\n{'='*80}")
print("Summary of Results")
print(f"{'='*80}")

for metric in metrics:
    print(f"\n📊 {metric}:")

    # Shapiro-Wilk汇总
    shapiro_df = all_results['shapiro'][metric]
    n_normal = (shapiro_df['P-value'] > 0.05).sum()
    n_total = len(shapiro_df)
    print(f"\n  Shapiro-Wilk Test:")
    print(f"    Normal distributions: {n_normal}/{n_total}")
    print(f"    Non-normal distributions: {n_total - n_normal}/{n_total}")

    # Wilcoxon汇总
    wilcoxon_df = all_results['wilcoxon'][metric]
    n_significant = (wilcoxon_df['P-value'] <= 0.05).sum()
    print(f"\n  Wilcoxon Signed-Rank Test:")
    print(f"    Significant differences: {n_significant}/{n_total}")
    print(f"    Non-significant differences: {n_total - n_significant}/{n_total}")

    # 列出显著差异的模型
    significant_models = wilcoxon_df[wilcoxon_df['P-value'] <= 0.05]['Comparison'].tolist()
    if significant_models:
        print(f"\n  Models with significant differences from TRMG:")
        for comp in significant_models:
            row = wilcoxon_df[wilcoxon_df['Comparison'] == comp].iloc[0]
            print(f"    - {comp}: {row['Interpretation']}")


Metric: RMSE

[Step 1] Shapiro-Wilk Normality Test for Differences
--------------------------------------------------------------------------------
Testing if (TRMG - Other_Model) follows normal distribution
H0: Data follows normal distribution
H1: Data does NOT follow normal distribution
Significance level: α = 0.05
--------------------------------------------------------------------------------

TRMG vs TRQG:
  Shapiro-Wilk Statistic: 0.888013
  P-value: 0.000000
  Distribution: Non-parametric (Not Normal)
  Reject H0 (not normal): Yes

TRMG vs WRGG:
  Shapiro-Wilk Statistic: 0.698055
  P-value: 0.000000
  Distribution: Non-parametric (Not Normal)
  Reject H0 (not normal): Yes

TRMG vs VRLG:
  Shapiro-Wilk Statistic: 0.648349
  P-value: 0.000000
  Distribution: Non-parametric (Not Normal)
  Reject H0 (not normal): Yes

TRMG vs ERMG:
  Shapiro-Wilk Statistic: 0.911216
  P-value: 0.000000
  Distribution: Non-parametric (Not Normal)
  Reject H0 (not normal): Yes

TRMG vs TSMG:
  Shapir

DM test

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from dieboldmariano import dm_test

In [9]:
def dm_test_all_models_quantiles(actual_path, pred_paths_dict, output_path):
    print("\nLoading actual values...")
    actual = pd.read_excel(actual_path)
    actual = np.squeeze(np.array(actual))
    print(f"  ✓ Loaded {len(actual)} data points")

    quantiles = np.arange(0.05, 1.0, 0.05)
    quantile_labels = [f'q{int(q*100):02d}' for q in quantiles]
    print(f"  Number of quantiles: {len(quantiles)}")

    baseline_model = 'TRMG'
    print("\nLoading model predictions...")
    model_data = {}
    for model_name, pred_path in pred_paths_dict.items():
        try:
            pred = pd.read_excel(pred_path)
            model_data[model_name] = np.array(pred)
            print(f"  ✓ {model_name}: shape {model_data[model_name].shape}")
        except Exception as e:
            print(f"  ✗ Error loading {model_name}: {e}")
            return None, None, None

    baseline_data = model_data[baseline_model]
    all_results = []

    print(f"\n{'='*80}")
    print(f"Running DM Tests: {baseline_model} vs Other Models")
    print(f"Using dieboldmariano.dm_test with Pinball Loss")
    print(f"{'='*80}\n")

    test_count = 0
    for model_name in pred_paths_dict.keys():
        if model_name == baseline_model:
            continue

        print(f"Testing {baseline_model} vs {model_name}...")
        comparison_data = model_data[model_name]

        for i, (q, q_label) in enumerate(zip(quantiles, quantile_labels)):
            try:
                baseline_pred = baseline_data[:, i]
                comparison_pred = comparison_data[:, i]

                # 定义pinball损失函数（注意：使用lambda捕获当前q值）
                pinball_loss_func = lambda v, p: q * (v - p) if v >= p else (q - 1) * (v - p)

                # 进行DM检验
                dm_stat, p_value = dm_test(actual, baseline_pred, comparison_pred,
                                           loss=pinball_loss_func, h=1)

                # 判断显著性
                is_significant = "Significant" if p_value <= 0.05 else "Not Significant"

                # 解释结果
                if p_value <= 0.05:
                    if dm_stat < 0:
                        performance = f"{baseline_model} significantly BETTER"
                        comparison_result = "TRMG Better"
                    else:
                        performance = f"{model_name} significantly BETTER"
                        comparison_result = "Other Better"
                else:
                    performance = "No significant difference"
                    comparison_result = "No Difference"

                # 计算平均损失差异以更好地理解性能差异
                loss_trmg = np.array([pinball_loss_func(v, p) for v, p in zip(actual, baseline_pred)])
                loss_other = np.array([pinball_loss_func(v, p) for v, p in zip(actual, comparison_pred)])
                mean_loss_diff = np.mean(loss_trmg - loss_other)
                avg_loss_trmg = np.mean(loss_trmg)
                avg_loss_other = np.mean(loss_other)

                all_results.append({
                    'Comparison': f'{baseline_model} vs {model_name}',
                    'Model_1': baseline_model,
                    'Model_2': model_name,
                    'Quantile': q,
                    'Quantile_Label': q_label,
                    'DM_Statistic': dm_stat,
                    'P_value': p_value,
                    'Mean_Loss_Diff': mean_loss_diff,
                    'Avg_Loss_TRMG': avg_loss_trmg,
                    'Avg_Loss_Other': avg_loss_other,
                    'Significance': is_significant,
                    'Comparison_Result': comparison_result,
                    'Reject_H0': 'Yes' if p_value <= 0.05 else 'No',
                    'Interpretation': performance
                })

                test_count += 1

            except Exception as e:
                print(f"  ✗ Error at quantile {q:.2f}: {e}")
                continue

        print(f"  ✓ Completed {len(quantiles)} quantile tests")

    # 检查是否有结果
    if len(all_results) == 0:
        print("\n✗ ERROR: No results generated!")
        return None, None, None

    print(f"\n✓ Total tests completed: {test_count}")

    # 转换为DataFrame
    results_df = pd.DataFrame(all_results)

    print(f"\nDataFrame created with {len(results_df)} rows and {len(results_df.columns)} columns")

    print(f"\n{'='*80}")
    print("Saving results to Excel...")
    print(f"{'='*80}\n")

    try:
        with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
            # Sheet 1: 所有结果
            results_df.to_excel(writer, sheet_name='All_Results', index=False)
            print("  ✓ Sheet 'All_Results' created")

            # Sheet 2-10: 每个对比模型一个sheet
            for model_name in pred_paths_dict.keys():
                if model_name == baseline_model:
                    continue

                model_results = results_df[results_df['Model_2'] == model_name].copy()
                if len(model_results) > 0:
                    sheet_name = f'{baseline_model}_vs_{model_name}'[:31]
                    model_results.to_excel(writer, sheet_name=sheet_name, index=False)
                    print(f"  ✓ Sheet '{sheet_name}' created")

            # Sheet 11: 按模型汇总
            model_summary_data = []
            for model_name in pred_paths_dict.keys():
                if model_name == baseline_model:
                    continue

                model_results = results_df[results_df['Model_2'] == model_name]
                if len(model_results) == 0:
                    continue

                total_tests = len(model_results)
                significant = (model_results['P_value'] <= 0.05).sum()
                trmg_better = (model_results['Comparison_Result'] == 'TRMG Better').sum()
                other_better = (model_results['Comparison_Result'] == 'Other Better').sum()
                no_diff = (model_results['Comparison_Result'] == 'No Difference').sum()

                model_summary_data.append({
                    'Model': model_name,
                    'Total_Comparisons': total_tests,
                    'Significant_Differences': significant,
                    'TRMG_Better': trmg_better,
                    'Other_Better': other_better,
                    'No_Significant_Difference': no_diff,
                    'Significance_Rate': f"{significant/total_tests*100:.1f}%",
                    'TRMG_Win_Rate': f"{trmg_better/total_tests*100:.1f}%" if total_tests > 0 else "0%"
                })

            if model_summary_data:
                model_summary_df = pd.DataFrame(model_summary_data)
                model_summary_df.to_excel(writer, sheet_name='Model_Summary', index=False)
                print("  ✓ Sheet 'Model_Summary' created")

            # Sheet 12: 按分位数汇总
            quantile_summary_data = []
            for q, q_label in zip(quantiles, quantile_labels):
                q_results = results_df[results_df['Quantile_Label'] == q_label]
                if len(q_results) == 0:
                    continue

                total_tests = len(q_results)
                significant = (q_results['P_value'] <= 0.05).sum()
                trmg_better = (q_results['Comparison_Result'] == 'TRMG Better').sum()
                other_better = (q_results['Comparison_Result'] == 'Other Better').sum()
                no_diff = (q_results['Comparison_Result'] == 'No Difference').sum()

                quantile_summary_data.append({
                    'Quantile': q,
                    'Quantile_Label': q_label,
                    'Total_Comparisons': total_tests,
                    'Significant_Differences': significant,
                    'TRMG_Better': trmg_better,
                    'Other_Better': other_better,
                    'No_Significant_Difference': no_diff,
                    'Significance_Rate': f"{significant/total_tests*100:.1f}%" if total_tests > 0 else "0%"
                })

            if quantile_summary_data:
                quantile_summary_df = pd.DataFrame(quantile_summary_data)
                quantile_summary_df.to_excel(writer, sheet_name='Quantile_Summary', index=False)
                print("  ✓ Sheet 'Quantile_Summary' created")

            # Sheet 13: 显著性结果矩阵（便于查看）
            # 创建透视表：行为分位数，列为模型，值为显著性结果
            pivot_df = results_df.pivot_table(
                index='Quantile_Label',
                columns='Model_2',
                values='Comparison_Result',
                aggfunc='first'
            )
            pivot_df.to_excel(writer, sheet_name='Result_Matrix')
            print("  ✓ Sheet 'Result_Matrix' created")

        print(f"\n✓ Results saved to: {output_path}")

        # 打印汇总统计
        print(f"\n{'='*80}")
        print("SUMMARY STATISTICS")
        print(f"{'='*80}\n")

        # 总体统计
        total_tests = len(results_df)
        significant_tests = (results_df['P_value'] <= 0.05).sum()
        trmg_better_total = (results_df['Comparison_Result'] == 'TRMG Better').sum()
        other_better_total = (results_df['Comparison_Result'] == 'Other Better').sum()

        print(f"📊 Overall Statistics:")
        print(f"  Total DM tests: {total_tests}")
        print(f"  Significant differences: {significant_tests} ({significant_tests/total_tests*100:.1f}%)")
        print(f"  TRMG better: {trmg_better_total} ({trmg_better_total/total_tests*100:.1f}%)")
        print(f"  Other models better: {other_better_total} ({other_better_total/total_tests*100:.1f}%)")

        # 按模型统计
        print(f"\n📊 Model-wise Statistics:")
        print(f"{'Model':<12} {'Sig/Total':<12} {'TRMG Wins':<12} {'Other Wins':<12} {'No Diff':<12}")
        print("-" * 70)

        for summary in model_summary_data:
            print(f"{summary['Model']:<12} "
                  f"{summary['Significant_Differences']}/{summary['Total_Comparisons']:<10} "
                  f"{summary['TRMG_Better']:<12} "
                  f"{summary['Other_Better']:<12} "
                  f"{summary['No_Significant_Difference']:<12}")

    except Exception as e:
        print(f"\n✗ Error saving Excel file: {e}")
        import traceback
        traceback.print_exc()
        return results_df, None, None

    return results_df, model_summary_df, quantile_summary_df

In [10]:
actual_path = 'C:/Users/ym_ya/Desktop/windpower/Simulation code and data/section 4/actual.xlsx'
pred_paths_dict = {'TRMG': 'C:/Users/ym_ya/Desktop/windpower/Simulation code and data/trmg/trmg.xlsx',
                   'TRQG': 'C:/Users/ym_ya/Desktop/windpower/Simulation code and data/trqg/trqg.xlsx',
                   'WRGG': 'C:/Users/ym_ya/Desktop/windpower/Simulation code and data/wrgg/wrgg.xlsx',
                   'VRLG': 'C:/Users/ym_ya/Desktop/windpower/Simulation code and data/vrlg/vrlg.xlsx',
                   'ERMG': 'C:/Users/ym_ya/Desktop/windpower/Simulation code and data/ermg/ermg.xlsx',
                   'TSMG': 'C:/Users/ym_ya/Desktop/windpower/Simulation code and data/tsmg/tsmg.xlsx',
                   'QRNN2': 'C:/Users/ym_ya/Desktop/windpower/Simulation code and data/qrnn2/qrnn2pred.xlsx',
                   'MCQRNN': 'C:/Users/ym_ya/Desktop/windpower/Simulation code and data/mcqrnn/mcqrnn2pred.xlsx',
                   'QRF': 'C:/Users/ym_ya/Desktop/windpower/Simulation code and data/qrf/predqrf.xlsx',
                   'QRLASSO': 'C:/Users/ym_ya/Desktop/windpower/Simulation code and data/qrlasso/lassopred.xlsx',}

output_path = '/Simulation code and data/5.2/dm_test_pinball_loss_results.xlsx'

results_df, quantile_summary, model_summary = dm_test_all_models_quantiles(
        actual_path, pred_paths_dict, output_path)


Loading actual values...
  ✓ Loaded 2621 data points
  Number of quantiles: 19

Loading model predictions...
  ✓ TRMG: shape (2621, 19)
  ✓ TRQG: shape (2621, 19)
  ✓ WRGG: shape (2621, 19)
  ✓ VRLG: shape (2621, 19)
  ✓ ERMG: shape (2621, 19)
  ✓ TSMG: shape (2621, 19)
  ✓ QRNN2: shape (2621, 19)
  ✓ MCQRNN: shape (2621, 19)
  ✓ QRF: shape (2621, 19)
  ✓ QRLASSO: shape (2621, 19)

Running DM Tests: TRMG vs Other Models
Using dieboldmariano.dm_test with Pinball Loss

Testing TRMG vs TRQG...
  ✓ Completed 19 quantile tests
Testing TRMG vs WRGG...
  ✓ Completed 19 quantile tests
Testing TRMG vs VRLG...
  ✓ Completed 19 quantile tests
Testing TRMG vs ERMG...
  ✓ Completed 19 quantile tests
Testing TRMG vs TSMG...
  ✓ Completed 19 quantile tests
Testing TRMG vs QRNN2...
  ✓ Completed 19 quantile tests
Testing TRMG vs MCQRNN...
  ✓ Completed 19 quantile tests
Testing TRMG vs QRF...
  ✓ Completed 19 quantile tests
Testing TRMG vs QRLASSO...
  ✓ Completed 19 quantile tests

✓ Total tests com